In [ ]:
import sys
import os
import math
# Navigate up one level to the parent directory and append it to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import nfl_data_py as nfl
import pandas as pd

from sklearn.linear_model import LogisticRegression
import plotly.express as px

from src import utils
from src import homers

# Research Question Pool

* could we allocate coefficients for voting?

* Were we homers? (picking our teams, picking the same teams)

* Was there correlation between doing well one week, and doing well the next week?

* What trends emerged (picking home/away, dogs/favorites)

* how did our underdog and SD picks fare against the spread

* Did the different lines matter?

In [ ]:
df = pd.read_pickle(homers.PROCESSED_FILE_PATH)

# Weekly Update

In [ ]:
# Filter for the rows with the highest season
season_df = df[df['season'] == df['season'].max()]
# Now, create a subset for the latest week in the highest season
week_df = season_df[season_df['week'] == season_df['week'].max()]
week_df.shape

In [ ]:
homers.plot_scores(week_df)

In [ ]:
homers.plot_scores(season_df, 'spread_pick', agg_sum=False)
latest_week = max(season_df[season_df['season'] == max(season_df['season'])]['week'])
last_4_week_criteria = season_df['week'] > latest_week - 4
homers.plot_scores(season_df[last_4_week_criteria], 'spread_pick', agg_sum=False)

In [ ]:
homers.plot_scores(df, 'spread_pick', agg_sum=False)

In [ ]:
homers.plot_scores(week_df, 'best_bet')

In [ ]:
homers.plot_scores(season_df, 'best_bet', agg_sum=False)

In [ ]:
homers.plot_scores(week_df, 'underdog_pick')

In [ ]:
homers.plot_scores(season_df, 'underdog_pick', agg_sum=False)
homers.plot_scores(df, 'underdog_pick', agg_sum=False)

In [ ]:
homers.plot_scores(week_df, 'survivor_pick')

# Consensus picks

* If we do a simple sum up of consensus picks, how did that score?
* What if we try a zero sum consensus?
* are the picks we are all on the same side on actually bad? majority vs no oposition? unanimous?

In [ ]:
SEASON, WEEK = 2023, 4
df = pd.read_pickle(homers.PROCESSED_FILE_PATH)

In [ ]:
# individual = df[(df['season']==SEASON) & (df['week'] == WEEK) & (df['picker'] != 'final') & (df['spread_pick']) & (~df['mnf_pick'])].copy()
individual = df[(df['picker'] != 'final') & (df['spread_pick']) & (~df['mnf_pick'])].copy()

In [ ]:
individual['pick_multiplier'] = individual['best_bet'] + 1

In [ ]:
individual

In [ ]:
consensus = individual.groupby(['pick', 'season', 'week'])[['spread_pick', 'best_bet']].sum().reset_index()
consensus['weighted_consensus'] = consensus['spread_pick'] + consensus['best_bet']
consensus = consensus.rename(columns={'spread_pick':'consensus'})
consensus = consensus.sort_values(by='weighted_consensus', ascending=False)
consensus = consensus.drop('best_bet', axis=1)
consensus

In [ ]:
df = pd.merge(consensus, individual[['season', 'week', 'pick', 'pick_result']], on=['season', 'week', 'pick'])
df = df.drop_duplicates()
df['pick_result'] = df.pick_result.apply(lambda x: math.ceil(math.floor(x) / 2))
df

In [ ]:
col = 'weighted_consensus'

In [ ]:
model = LogisticRegression()
model.fit(df[[col]], df['pick_result'])
print(model.coef_[0][0])
math.exp(model.coef_[0][0])

In [ ]:
df.groupby(col).pick_result.count()

In [ ]:
df.groupby(col).pick_result.mean()

In [ ]:
fig = px.bar(
    df.groupby(col).pick_result.count(),
    text_auto=True,
    title=f'Pick count by {col} score'
)
fig.show()

In [ ]:
fig = px.bar(
    df.groupby(col).pick_result.mean().round(2),
    text_auto=True,
    title=f'Pick % by {col} score'
)
fig.show()

In [ ]:
df

In [ ]:
schedule = nfl.import_schedules([SEASON])
week_schedule = schedule[schedule['week']==WEEK]
week_schedule

In [ ]:
opponent_dict = dict(**dict(zip(week_schedule['away_team'], week_schedule['home_team'])), **dict(zip(week_schedule['home_team'], week_schedule['away_team'])))
consensus['opponent'] = consensus['pick'].map(opponent_dict)
consensus = pd.merge(consensus, consensus[['spread_pick', 'best_bet', 'weighted_consensus', 'opponent']], left_on='pick', right_on='opponent', suffixes=("", '_opponent'))
consensus = consensus.drop(columns=['opponent', 'pick_opponent'])
consensus

In [ ]:
pd.merge(consensus, individual, left_index=True, right_on='pick')

# Team Pick Frequency & Homerism

In [ ]:
spread_picks = df[df['spread_pick'] & (df['season'] == 2023)].copy()
spread_picks['win'] = spread_picks['pick_result'] >= 1
spread_picks

In [ ]:
for picker in spread_picks['picker'].unique():
    tmp = (
        spread_picks[spread_picks['picker'] == str(picker)]
        .groupby('pick')['pick']
        .count()
        .sort_values(ascending=False)
        .to_frame()
        .rename(columns={'pick': 'times_picked'})
        .reindex(utils.nfl_teams, fill_value=0)
        .reset_index()
        .rename(columns={'pick': 'team'})
    )
       
    col = 'times_picked'

    # Create the bar chart
    fig = px.bar(tmp, x=col, y='team', orientation='h', color='team', color_discrete_map=utils.team_unique_colors)


    tmp2 = spread_picks[spread_picks['picker'] == picker].groupby('pick')['best_bet'].sum().sort_values(ascending=False).to_frame().rename(columns={'pick': 'times_picked'}).reset_index().rename(columns={'pick': 'team'})
    tmp2['color'] = tmp2['team'].map(utils.team_unique_colors)
    
    tmp = pd.merge(tmp, tmp2, how='left', on='team').fillna(0)
    
    # Duplicate the data to create the stacked bar with lower opacity
    fig.add_bar(x=tmp['best_bet'], y=tmp['team'], orientation='h', marker_color=tmp['color'], opacity=0.5)

    fig.update_yaxes(categoryorder='total ascending')
    fig.update_layout(
        height=1000,
        width=800,
        xaxis_title=f'Times Picked ATS',
        yaxis_title='Teams',
        title=f'Times {picker.capitalize()} Picked a Team ATS',
    )
    fig.update_traces(showlegend=False)

    # Iterate through the data and add logos to the chart
    for index, row in tmp.iterrows():
        team = row['team']
        scale = 1.25
        fig.add_layout_image(
            dict(source=f'https://a.espncdn.com/i/teamlogos/nfl/500/{team}.png',
                x=row[col]+row['best_bet'],  # Adjust the position
                y=team,
                xref="x",
                yref="y",
                sizex=scale,  # Adjust the size
                sizey=scale,  # Adjust the size
                sizing="contain",
                opacity=1,
                xanchor="center",
                yanchor="middle",        
            )
        )

    # Show the chart
    fig.show()

## How many times do we pick against teams?

In [ ]:
find_opponent = lambda row: row.home_team if row.away_team == row.pick else row.away_team
spread_picks['opponent'] = spread_picks.apply(find_opponent, axis=1)
spread_picks.head()

In [ ]:
for picker in spread_picks['picker'].unique():
    tmp = (
        spread_picks[spread_picks['picker'] == str(picker)]
        .groupby('opponent')['opponent']
        .count()
        .sort_values(ascending=False)
        .to_frame()
        .rename(columns={'opponent': 'times_picked_against'})
        .reindex(utils.nfl_teams, fill_value=0)
        .reset_index()
        .rename(columns={'opponent': 'team'})
    )
    
    col = 'times_picked_against'

    # Create the bar chart
    fig = px.bar(tmp, x=col, y='team', orientation='h', color='team', color_discrete_map=utils.team_unique_colors)
    
    tmp2 = (
        spread_picks[spread_picks['picker'] == picker]
        .groupby('opponent')['best_bet']
        .sum()
        .sort_values(ascending=False)
        .to_frame()
        .rename(columns={'opponent': 'times_picked_against'})
        .reset_index()
        .rename(columns={'opponent': 'team'})
    )
    
    tmp2['color'] = tmp2['team'].map(utils.team_unique_colors)
    
    tmp = pd.merge(tmp, tmp2, how='left', on='team').fillna(0)
    
    # Duplicate the data to create the stacked bar with lower opacity
    fig.add_bar(x=tmp['best_bet'], y=tmp['team'], orientation='h', marker_color=tmp['color'], opacity=0.5)

    fig.update_yaxes(categoryorder='total ascending')
    fig.update_layout(
        height=1000,
        width=800,
        xaxis_title=f'Times Picked Against',
        yaxis_title='Teams',
        title=f'Times {picker.capitalize()} Picked Against a Team ATS',
    )
    fig.update_traces(showlegend=False)

    # Iterate through the data and add logos to the chart
    for index, row in tmp.iterrows():
        team = row['team']
        scale = 1.25
        fig.add_layout_image(
            dict(source=f'https://a.espncdn.com/i/teamlogos/nfl/500/{team}.png',
                x=row[col]+row['best_bet'],  # Adjust the position
                y=team,
                xref="x",
                yref="y",
                sizex=scale,  # Adjust the size
                sizey=scale,  # Adjust the size
                sizing="contain",
                opacity=1,
                xanchor="center",
                yanchor="middle",        
            )
        )

    # Show the chart
    fig.show()

In [ ]:
picker = 'griffin'

against_tmp = (
    spread_picks[spread_picks['picker'] == picker]
    .groupby('opponent')['opponent']
    .count()
    .sort_values(ascending=False)
    .to_frame()
    .rename(columns={'opponent': 'times_picked_against'})
    .reindex(utils.nfl_teams, fill_value=0)
    .reset_index()
    .rename(columns={'opponent': 'team'})
)

col = 'times_picked_against'


against_tmp2 = (
    spread_picks[spread_picks['picker'] == picker]
    .groupby('opponent')['best_bet']
    .sum()
    .sort_values(ascending=False)
    .to_frame()
    .rename(columns={'opponent': 'times_picked_against'})
    .reset_index()
    .rename(columns={'opponent': 'team'})
)

# against_tmp2['color'] = against_tmp2['team'].map(utils.team_unique_colors)

against_tmp = pd.merge(against_tmp, against_tmp2, how='left', on='team').fillna(0)

against_tmp['times_picked_against'] = against_tmp['times_picked_against'] + against_tmp['best_bet'].astype('int')
against_tmp = against_tmp.drop(columns=['best_bet'])
against_tmp

In [ ]:
picker = 'griffin'

tmp = (
    spread_picks[spread_picks['picker'] == picker]
    .groupby('pick')['pick']
    .count()
    .sort_values(ascending=False)
    .to_frame()
    .rename(columns={'pick': 'times_picked'})
    .reindex(utils.nfl_teams, fill_value=0)
    .reset_index()
    .rename(columns={'pick': 'team'})
)
    
col = 'times_picked'


tmp2 = (
    spread_picks[spread_picks['picker'] == picker]
    .groupby('pick')['best_bet']
    .sum()
    .sort_values(ascending=False)
    .to_frame()
    .rename(columns={'pick': 'times_picked'})
    .reset_index()
    .rename(columns={'pick': 'team'})
)
# tmp2['color'] = tmp2['team'].map(utils.team_unique_colors)

tmp = pd.merge(tmp, tmp2, how='left', on='team').fillna(0)

tmp['times_picked'] = tmp['times_picked'] + tmp['best_bet'].astype('int')
tmp = tmp.drop(columns=['best_bet'])
tmp

# against_tmp['times_picked_against'] = against_tmp['times_picked_against'] + against_tmp['best_bet'].astype('int')
# against_tmp = against_tmp.drop(columns=['best_bet'])
# against_tmp

In [ ]:
combined = pd.merge(tmp, against_tmp)
combined['color'] = combined['team'].map(utils.team_unique_colors)

fig = px.scatter(combined, x='times_picked', y='times_picked_against', color='team', color_discrete_map=utils.team_unique_colors)

fig.update_layout(
    height=600,
    width=800,
    xaxis_title=f'Times Picked',
    yaxis_title='Times',
    title=f'Times {picker.capitalize()} Picked Against a Team ATS',
)
fig.update_traces(showlegend=False)

# Iterate through the data and add logos to the chart
for index, row in tmp.iterrows():
    team = row['team']
    scale = 1.25
    fig.add_layout_image(
        dict(source=f'https://a.espncdn.com/i/teamlogos/nfl/500/{team}.png',
            x=row[col]+row['best_bet'],  # Adjust the position
            y=team,
            xref="x",
            yref="y",
            sizex=scale,  # Adjust the size
            sizey=scale,  # Adjust the size
            sizing="contain",
            opacity=1,
            xanchor="center",
            yanchor="middle",        
        )
    )

# Graveyard

what do i need to do:
* find the most popular picks
* make a list of them with "consensus" as the picker and the other necessary cols
* join in nfl data and evaluate


In [ ]:
nfl_df = nfl.import_schedules([SEASON])[['game_id', 'season', 'week', 'away_team', 'home_team', 'away_score', 'home_score', 'result', 'spread_line']] 




# turn pick types into one hot cols to make lookup faster
transformed['spread_pick'] = transformed['pick_type'].map({'ud': False, 'sd': False}).fillna(True)
transformed['best_bet'] = transformed['pick_type'] == 'bb'
transformed['underdog_pick'] = transformed['pick_type'] == 'ud'
transformed['survivor_pick'] = transformed['pick_type'] == 'sd'
transformed['mnf_pick'] = transformed['pick_type'] == 'mnf'
transformed = transformed.drop(columns='pick_type')

# join home and away picks
joined_away = pd.merge(
    transformed, 
    nfl_df,
    left_on=['season', 'week', 'pick'],
    right_on=['season', 'week', 'away_team']
)
joined_home = pd.merge(
    transformed, 
    nfl_df,
    left_on=['season', 'week', 'pick'],
    right_on=['season', 'week', 'home_team']
)


week_final_picks = pd.concat([joined_home, joined_away], ignore_index=True)
week_final_picks['away_pick'] = week_final_picks['pick'] == week_final_picks['away_team']
week_final_picks['away_cover'] = week_final_picks.apply(utils.cover_result, axis=1)
week_final_picks['home_cover'] = 1 - week_final_picks['away_cover']
week_final_picks['pick_result'] = week_final_picks.apply(utils.pick_result, axis=1)

